In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
import os
import sys
import pickle

from utilities3 import *
from sklearn.model_selection import train_test_split
from Adam import Adam
from timeit import default_timer

# import local modules from FNO2D.py
import FNO2D

In [2]:
# read data from csv file
gc = pd.read_csv("Data/gc_samples_filtered.csv", header=None)
d = pd.read_csv("Data/d_samples_filtered.csv", header=None)

In [3]:
# check number of samples in gc
print("Number of samples in gc: ", len(gc))

Number of samples in gc:  3207


In [4]:
# coordinates data from csv file
coordinates = pd.read_csv("Data/coordinates_n", header=None)

# output mesh coordinates
damage_x = pd.read_csv('Data/x_ver', header=None)
damage_y = pd.read_csv('Data/y_ver', header=None)


In [5]:
print("x_ver shape: ", damage_x.shape)
print("y_ver shape: ", damage_y.shape)

x_ver shape:  (3918, 1)
y_ver shape:  (3918, 1)


In [6]:
# concatenate damage_x and damage_y
damage_xy = np.concatenate((damage_x, damage_y), axis=1)

In [7]:
#  convert to torch tensor
print("gc shape: ", gc.shape)
print("d shape: ", d.shape)
print("coordinates shape: ", coordinates.shape)
print("damage_xy shape: ", damage_xy.shape)

gc shape:  (3207, 7610)
d shape:  (3207, 3918)
coordinates shape:  (7610, 2)
damage_xy shape:  (3918, 2)


In [8]:
# create numpy array with shape (len(gc), gc.shape[1], 2)
input_mesh = np.zeros((len(gc), gc.shape[1], 2))
damage_mesh = np.zeros((len(gc), damage_xy.shape[0], 2))

# for each sample in input_mesh, add the coordinates
for i in range(len(gc)):
    input_mesh[i, :, :] = coordinates
    damage_mesh[i, :, :] = damage_xy[i, :]
    
# add gc to the last dimension of input_mesh to have shape of (len(gc), gc.shape[1], 3)
input_data = np.concatenate((input_mesh, np.expand_dims(gc, axis=2)), axis=2)
input_data = torch.from_numpy(input_data).float()

damage_data = np.concatenate((damage_mesh, np.expand_dims(d, axis=2)), axis=2)
damage_data = torch.from_numpy(damage_data).float()

# damage_mesh = torch.from_numpy(damage_mesh).float()
# input_mesh = torch.from_numpy(input_mesh).float()

In [9]:
# config for model

batch_size = 8
learning_rate = 0.001

epochs = 501
step_size = 50
gamma = 0.5

modes = 12
width = 32

In [10]:
# train test split X data, for use in creating TensorDataset
X_train, X_test, y_train, y_test = train_test_split(input_data, damage_data, test_size=0.2, random_state=42)

train_loader = DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(torch.utils.data.TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False)


In [21]:
import importlib
importlib.reload(FNO2D)

<module 'FNO2D' from '/mnt/c/Users/Jin Yi/Documents/ArianaPHD/research/neural_operator/FNO2D.py'>

In [22]:
model = FNO2D.FNO2d(modes, modes, width=width, in_channels=3, out_channels=1).cuda()
model_iphi = FNO2D.IPHI(width=width).cuda()

In [23]:
print(count_params(model), count_params(model_iphi))

params = list(model.parameters()) + list(model_iphi.parameters())
optimizer = Adam(params, lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

1482689 63746


In [24]:
X_train.shape[0]

2565

In [25]:
input_data[:, :, :2].shape

torch.Size([3207, 7610, 2])

In [26]:
myloss = LpLoss(size_average=False)
N_sample = 1000 # number of samples for regularization
for ep in range(epochs):
    model.train()
    t1 = default_timer()
    train_l2 = 0
    train_reg = 0
    for data, damage in train_loader:
        damage, data = damage.cuda(), data.cuda()
        # samples_x = torch.rand(batch_size, N_sample, 3).cuda() * 3 -1

        damage_values = damage[:, :, 2]

        optimizer.zero_grad()
        out = model(data, iphi=model_iphi, x_in = data[:, :, :2], x_out = damage[:, :, :2])
        # print(out.shape)
        # samples_xi = model_iphi(samples_x)

        # if not last batch, use batch_size, else use the remainder
        if len(damage_values) == batch_size:
            loss_data = myloss(out.view(batch_size, -1), damage_values.view(batch_size, -1))
        else:
            loss_data = myloss(out.view(len(damage_values), -1), damage_values.view(len(damage_values), -1))
        # loss_reg = myloss(samples_xi, samples_x)
        # loss = loss_data + 0.000 * loss_reg
        loss = loss_data + 0.000
        loss.backward()

        optimizer.step()
        train_l2 += loss_data.item()
        # train_reg += loss_reg.item()

    scheduler.step()

    model.eval()
    test_l2 = 0.0
    with torch.no_grad():
        for data, damage in test_loader:
            data, damage = data.cuda(), damage.cuda()
            # out = model(mesh, iphi=model_iphi)
            damage_values_test = damage[:, :, 2]
            out = model(data, iphi=model_iphi, x_in = data[:, :, :2], x_out = damage[:, :, :2])
            # if not last batch, use batch_size, else use the remainder
            if len(damage_values_test) == batch_size:
                test_l2 += myloss(out.view(batch_size, -1), damage_values_test.view(batch_size, -1)).item()
            else:
                test_l2 += myloss(out.view(len(damage_values_test), -1), damage_values_test.view(len(damage_values_test), -1)).item()

    train_l2 /= X_train.shape[0]
    # train_reg /= X_train.shape[0]
    test_l2 /= X_test.shape[0]

    t2 = default_timer()
    print(ep, t2 - t1, train_l2, test_l2)

    if ep%100==0:
        XY = data[-1].squeeze().detach().cpu().numpy()
        truth = damage[-1].squeeze().detach().cpu().numpy()
        pred = out[-1].squeeze().detach().cpu().numpy()

        # lims = dict(cmap='RdBu_r', vmin=truth.min(), vmax=truth.max())
        # fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
        # ax[0].scatter(XY[:, 0], XY[:, 1], 100, truth, edgecolor='w', lw=0.1, **lims)
        # ax[1].scatter(XY[:, 0], XY[:, 1], 100, pred, edgecolor='w', lw=0.1, **lims)
        # ax[2].scatter(XY[:, 0], XY[:, 1], 100, truth - pred, edgecolor='w', lw=0.1, **lims)
        # fig.show()


0 23.459415493067354 1.815806576615421 0.9083900696763368
1 23.326877016108483 0.9085984008818807 0.9081332464455816
2 23.375878255348653 0.908978809297201 0.9096143048871714
3 23.369796778075397 0.9085510887830114 0.9088523183656259


In [26]:
# clear cuda cache and memory
torch.cuda.empty_cache()

In [47]:
print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')


Allocated: 5.0 GB
Cached:    5.3 GB


/home/jinyi12/miniconda3/envs/phasefield/lib/python3.8/site-packages/torch/cuda/memory.py:384: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(
